<a href="https://colab.research.google.com/github/Nathalia1234/Tarefa-2.6---Roteiro_CNN_-_Transferlearning/blob/main/Tarefa_2_6_Treinamento_FINE_TUNNING(train_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Pré-processamento das Imagens
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Defina o número de classes
num_classes = 10  # Altere para o número real de classes no dataset

# Data augmentation e normalização para dados de treino
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apenas normalização para dados de validação
validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Geração de dados para treino e validação a partir de diretórios
train_generator = train_datagen.flow_from_directory(
    'path/to/train',  # Substitua pelo caminho da pasta de treino
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    'path/to/validation',  # Substitua pelo caminho da pasta de validação
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Obtenha o Modelo Pré-treinado e Realize o Fine-Tuning
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Carregar o modelo pré-treinado sem a camada de classificação no topo
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas da base
for layer in base_model.layers:
    layer.trainable = False

# Adicionar novas camadas de classificação
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento do Modelo
batch_size = 32
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20
)

# Salvar o modelo treinado
model.save('best_model.h5')

# Avaliação e Gráficos de Métricas
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

# Gráficos de Loss e Acurácia
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Loss de Treinamento')
plt.plot(history.history['val_loss'], label='Loss de Validação')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Predições e métricas de validação
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes

# F1 Score, Precisão e Recall
f1 = f1_score(y_true, y_pred_classes, average='weighted')
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f"F1 Score: {f1}, Precisão: {precision}, Recall: {recall}")

# Matriz de Confusão
cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=validation_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train'